In [1]:
from music21 import *
import tensorflow as tf
from tensorflow.keras import *

from tensorflow.keras.layers import *
from tensorflow.keras.models import *
from tensorflow.keras.callbacks import *
import tensorflow.keras.backend as K

from collections import Counter
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import numpy as np
import os

# Load the TensorBoard notebook extension.
%load_ext tensorboard

In [2]:
# path to midi files
midi_path = "..\\..\\data\\videogame_midi_mini"
midi_dir = os.listdir(midi_path)
len(midi_dir)

5

In [3]:
# with this videogame data set, we have to change this code for the possibility of non-piano notes in this midi set
def read_midi(file):
    
    print("Loading Music File:",file)
    
    notes=[]
    notes_to_parse = None
    
    #parsing a midi file
    midi = converter.parse(file)
  
    #grouping based on different instruments
    s2 = instrument.partitionByInstrument(midi)

    #Looping over all the instruments
    for part in s2.parts:
    
        #select elements of only piano
        if 'Piano' in str(part): 
        
            notes_to_parse = part.recurse() 
      
            #finding whether a particular element is note or a chord
            for element in notes_to_parse:
                
                #note
                if isinstance(element, note.Note):
                    notes.append(str(element.pitch))
                
                #chord
                elif isinstance(element, chord.Chord):
                    notes.append('.'.join(str(n) for n in element.normalOrder))

    return np.array(notes)

In [4]:
# create array of midi file notes
midi_array = np.array([read_midi(os.path.join(midi_path, f)) for f in midi_dir])

Loading Music File: ..\..\data\videogame_midi_mini\ff1battp.mid
Loading Music File: ..\..\data\videogame_midi_mini\FF3_Battle_%28Piano%29.mid
Loading Music File: ..\..\data\videogame_midi_mini\FF3_Third_Phase_Final_%28Piano%29.mid
Loading Music File: ..\..\data\videogame_midi_mini\ff4-fight1.mid
Loading Music File: ..\..\data\videogame_midi_mini\Ff7-One_Winged.mid


In [5]:
#converting 2D array into 1D array
notes_ = [element for note_ in midi_array for element in note_]

#No. of unique notes
unique_notes = list(set(notes_))
print(len(unique_notes))

159


In [6]:
# same as v2 but not redacting notes
freq = dict(Counter(notes_))

frequent_notes = [note_ for note_, count in freq.items() if count>=1]
print(len(frequent_notes))

159


In [7]:
new_music=[]

for notes in midi_array:
    temp=[]
    for note_ in notes:
        if note_ in frequent_notes:
            temp.append(note_)            
    new_music.append(temp)
    
midi_array = np.array(new_music)

In [8]:
# data prep
# changing the amountt of timesteps to be 16, will allow for predictor to change must faster during generative process,
# allegedly
no_of_timesteps = 16
x = []
y = []

for note_ in midi_array:
    for i in range(0, len(note_) - no_of_timesteps, 1):
        
        #preparing input and output sequences
        input_ = note_[i:i + no_of_timesteps]
        output = note_[i + no_of_timesteps]
        
        x.append(input_)
        y.append(output)
        
x = np.array(x)
y = np.array(y)

In [9]:
# assign unique int to each note
unique_x = list(set(x.ravel()))
x_note_to_int = dict((note_, number) for number, note_ in enumerate(unique_x))

In [10]:
#preparing input sequences
x_seq=[]
for i in x:
    temp = []
    for j in i:
        #assigning unique integer to every note
        temp.append(x_note_to_int[j])
    x_seq.append(temp)
    
x_seq = np.array(x_seq)

In [11]:
unique_y = list(set(y))
y_note_to_int = dict((note_, number) for number, note_ in enumerate(unique_y)) 
y_seq=np.array([y_note_to_int[i] for i in y])

In [12]:
x_tr, x_val, y_tr, y_val = train_test_split(x_seq,y_seq,test_size=0.2,random_state=0)
x_tr.shape

(2972, 16)

In [13]:
# let's just save the version with the lowest validation loss this time around
mc = callbacks.ModelCheckpoint(
    'vgg_mini_midis.hdf5',
    monitor='val_loss',
    mode='min',
    save_best_only = True,
    verbose=1
)

In [14]:
K.clear_session()
model = Sequential()
    
#embedding layer
model.add(Embedding(len(unique_x), 100, input_length = 16,trainable = True)) 

model.add(Conv1D(32,3, padding='causal', activation='relu'))
model.add(Dropout(0.2))
model.add(MaxPool1D(2))
    
model.add(Conv1D(64, 3, activation = 'relu', dilation_rate = 2, padding = 'causal'))
model.add(Dropout(0.2))
model.add(MaxPool1D(2))

model.add(Conv1D(128, 3, activation = 'relu', dilation_rate = 4, padding = 'causal'))
model.add(Dropout(0.2))
model.add(MaxPool1D(2))
            
model.add(GlobalMaxPool1D())
    
model.add(Dense(128, activation = 'relu'))
model.add(Dense(len(unique_y), activation = 'softmax'))
    
model.compile(loss = 'sparse_categorical_crossentropy', optimizer = 'adam')

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 16, 100)           15900     
_________________________________________________________________
conv1d (Conv1D)              (None, 16, 32)            9632      
_________________________________________________________________
dropout (Dropout)            (None, 16, 32)            0         
_________________________________________________________________
max_pooling1d (MaxPooling1D) (None, 8, 32)             0         
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 8, 64)             6208      
_________________________________________________________________
dropout_1 (Dropout)          (None, 8, 64)             0         
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 4, 64)             0

In [17]:
history = model.fit(
    np.array(x_tr),
    np.array(y_tr),
    batch_size=128,
    epochs=500,
    validation_data=(
        np.array(x_val),
        np.array(y_val)
    ),
    verbose=1,
    callbacks=[mc]
)

Train on 2972 samples, validate on 744 samples
Epoch 1/500
 128/2972 [>.............................] - ETA: 13sWARNING:tensorflow:Can save best model only with val_loss available, skipping.


UnknownError: 2 root error(s) found.
  (0) Unknown:  Failed to get convolution algorithm. This is probably because cuDNN failed to initialize, so try looking to see if a warning log message was printed above.
	 [[node sequential/conv1d/conv1d (defined at <ipython-input-15-d85ea24c7dab>:11) ]]
	 [[Reshape_30/_36]]
  (1) Unknown:  Failed to get convolution algorithm. This is probably because cuDNN failed to initialize, so try looking to see if a warning log message was printed above.
	 [[node sequential/conv1d/conv1d (defined at <ipython-input-15-d85ea24c7dab>:11) ]]
0 successful operations.
0 derived errors ignored. [Op:__inference_distributed_function_1599]

Function call stack:
distributed_function -> distributed_function


In [ ]:
%tensorboard --logdir logs/scalars

In [ ]:
#loading best model
model = models.load_model('schubert_midis.hdf5')

In [ ]:
import random
# create list of notes
predictions=[]



# create something x notes long
for i in range(16):
    # take random starting note from x_validation set
    ind = np.random.randint(0,len(x_val)-1)
    random_music = x_val[ind]
    # change vector to match dimension of timesteps
    random_music = random_music.reshape(1,no_of_timesteps)

    # predict a note for the given x model, and append maximum prob value note
    prob  = model.predict(random_music)[0]
    y_pred= np.argmax(prob,axis=0)
    predictions.append(y_pred)

    random_music = np.insert(random_music[0],len(random_music[0]),y_pred)
    random_music = random_music[1:]
    
print(predictions)

In [ ]:
x_int_to_note = dict((number, note_) for number, note_ in enumerate(unique_x)) 
predicted_notes = [x_int_to_note[i] for i in predictions]
print(predicted_notes)

In [ ]:
def convert_to_midi(prediction_output, name):
    offset = 0
    output_notes = []

    # create note and chord objects based on the values generated by the model
    for pattern in prediction_output:
        
        # pattern is a chord
        if ('.' in pattern) or pattern.isdigit():
            notes_in_chord = pattern.split('.')
            notes = []
            for current_note in notes_in_chord:
                
                cn=int(current_note)
                new_note = note.Note(cn)
                new_note.storedInstrument = instrument.Piano()
                notes.append(new_note)
                
            new_chord = chord.Chord(notes)
            new_chord.offset = offset
            output_notes.append(new_chord)
            
        # pattern is a note
        else:  
            new_note = note.Note(pattern)
            new_note.offset = offset
            new_note.storedInstrument = instrument.Piano()
            output_notes.append(new_note)

        # increase offset each iteration so that notes do not stack
        offset += 1
    midi_stream = stream.Stream(output_notes)
    midi_stream.write('midi', fp='{}.mid'.format(name))

In [ ]:
convert_to_midi(predicted_notes, "vg_mini_1")